In [4]:
import numpy as np
import pandas as pd
import os
import glob

In [5]:
msg_paths = sorted(glob.glob("../data/results/*.csv"))

df = pd.read_csv(msg_paths[0])
df

,timestamp,TimeUS,Roll,Pitch,Yaw,Alt,Lat,Lng,Q1,Q2,Q3,Q4,label
0,2024-11-12 14:02:01,287589565,0.88,0.73,78.79,70.110001,35.928239,128.871155,0.772733,0.001907,0.009879,0.634652,0
1,2024-11-12 14:02:01,287688106,0.88,0.73,78.79,70.199997,35.928239,128.871155,0.772718,0.001856,0.009852,0.634671,0
2,2024-11-12 14:02:02,287788037,0.87,0.73,78.79,70.099998,35.928239,128.871155,0.772720,0.001804,0.009811,0.634669,0
3,2024-11-12 14:02:02,287888029,0.87,0.73,78.79,70.290001,35.928239,128.871155,0.772763,0.001860,0.009781,0.634617,0
4,2024-11-12 14:02:02,287988089,0.88,0.72,78.78,70.139999,35.928239,128.871155,0.772776,0.001957,0.009758,0.634601,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
125966,2024-11-15 14:50:43,1370091355,0.34,-0.06,253.20,72.439995,35.928328,128.871678,0.596299,0.001361,-0.002779,-0.802756,1
125967,2024-11-15 14:50:43,1370191390,0.35,-0.06,253.23,72.610001,35.928328,128.871678,0.596457,0.001387,-0.002793,-0.802639,1
125968,2024-11-15 14:50:43,1370291411,0.35,-0.05,253.24,72.639999,35.928328,128.871678,0.596554,0.001480,-0.002749,-0.802567,1
125969,2024-11-15 14:50:43,1370391419,0.35,-0.05,253.26,72.659996,35.928328,128.871678,0.596663,0.001463,-0.002817,-0.802486,1


In [6]:
# from pycaret.classification import *

# df = pd.read_csv(msg_paths[1])
# n_df = df.drop(["timestamp", "TimeUS"], axis=1)

# clf1 = setup(data=n_df, target='label', session_id=42, verbose=False)

# tree_models = ['rf', 'et', 'dt', 'ada', 'gbc']
# best_model = compare_models(include=tree_models)

# sample_df = df.sample(n=3000, random_state=42)  # 3000~5000개로 충분

# # 4. 모델 해석 (SHAP 기반)
# # interpret_model(best_model)

# # 5. 평가 지표 확인
# # evaluate_model(best_model)


In [ ]:
from pycaret.classification import *
import shap
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 전체 결과를 저장할 리스트
all_shap_results = []

for i in range(len(msg_paths)):
    msg_name = os.path.basename(msg_paths[i]).split('_')[0]
    print(f"{msg_name} Tree모델 학습 및 Best Model SHAP 분석")

    try:
        # 데이터 로딩 및 정제
        df = pd.read_csv(msg_paths[i])
        n_df = df.drop(["timestamp", "TimeUS"], axis=1)

        # PyCaret 학습
        clf1 = setup(data=n_df, target='label', session_id=42, verbose=False)
        tree_models = ['rf', 'et', 'dt', 'ada', 'gbc']
        best_model = compare_models(include=tree_models, sort="F1")

        # 클래스 균형 샘플링
        normal_sample = n_df[n_df['label'] == 0].sample(n=1500, random_state=42)
        anomaly_sample = n_df[n_df['label'] == 1].sample(n=1500, random_state=42)
        sample_df = pd.concat([normal_sample, anomaly_sample])
        X = sample_df.drop(columns=['label'])

        # SHAP 계산
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X)

        if isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
            shap_array = shap_values[:, :, 1]
        elif isinstance(shap_values, list):
            shap_array = np.array(shap_values[1])
        else:
            shap_array = np.array(shap_values)

        # 중요도 계산 및 필드 이름 추가
        shap_importance = pd.DataFrame({
            'feature': X.columns,
            'mean_abs_shap': np.abs(shap_array).mean(axis=0)
        })
        shap_importance['field'] = msg_name
        # shap_importance = shap_importance.sort_values(by='mean_abs_shap', ascending=False)
        shap_importance['shap_ratio'] = shap_importance['mean_abs_shap'] / shap_importance['mean_abs_shap'].sum()
        shap_importance = shap_importance[['field', 'feature', 'shap_ratio']]

        # 중요도 기준 정렬
        shap_importance = shap_importance.sort_values(by='shap_ratio', ascending=False)

        # 누적 저장
        all_shap_results.append(shap_importance)

    except Exception as e:
        print(f"XXXXX {msg_name} 분석 실패: {e} XXXXX")
        continue

# 전체 통합 테이블 생성 및 저장
all_shap_df = pd.concat(all_shap_results, ignore_index=True)
os.makedirs("shap_results", exist_ok=True)
all_shap_df.to_csv("shap_results/all_fields_shap_importance_ratio.csv", index=False)
print("\n 모든 필드 SHAP 중요도 통합 저장 완료: shap_results/all_fields_shap_importance_ratio.csv")



AHR2 Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7140
et,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.6670
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3600
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.7310
dt,Decision Tree Classifier,0.9999,0.9999,0.9999,0.9999,0.9999,0.9998,0.9998,0.0470


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

ATT Tree모델 학습 및 Best Model SHAP 분석


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:02:47
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
from pycaret.classification import *
import shap
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# 전체 결과를 저장할 리스트
all_shap_results = []
all_model_results = []


for i in range(len(msg_paths)):
    msg_name = os.path.basename(msg_paths[i]).split('_')[0]
    print(f"{msg_name} Tree모델 학습 및 Best Model SHAP 분석")

    try:
        # 데이터 로딩 및 정제
        df = pd.read_csv(msg_paths[i])
        n_df = df.drop(["timestamp", "TimeUS"], axis=1)

        # PyCaret 학습
        clf1 = setup(data=n_df, target='label', session_id=42, verbose=False)
        tree_models = ['rf', 'et', 'dt', 'ada', 'gbc']
        best_model = compare_models(include=tree_models, sort="F1")

        # 모델 성능 테이블 추출 및 필드명 추가
        model_result = pull()
        model_result['field'] = msg_name
        model_result = model_result[['field', 'Model', 'Accuracy', 'AUC', 'Recall', 'Prec.', 'F1', 'Kappa', 'MCC', 'TT (Sec)']]
        all_model_results.append(model_result)


        # 클래스 균형 샘플링
        normal_sample = n_df[n_df['label'] == 0].sample(n=1500, random_state=42)
        anomaly_sample = n_df[n_df['label'] == 1].sample(n=1500, random_state=42)
        sample_df = pd.concat([normal_sample, anomaly_sample])
        X = sample_df.drop(columns=['label'])

        # SHAP 계산
        explainer = shap.TreeExplainer(best_model)
        shap_values = explainer.shap_values(X)

        if isinstance(shap_values, np.ndarray) and shap_values.ndim == 3:
            shap_array = shap_values[:, :, 1]
        elif isinstance(shap_values, list):
            shap_array = np.array(shap_values[1])
        else:
            shap_array = np.array(shap_values)

        # 중요도 계산 및 필드 이름 추가
        shap_importance = pd.DataFrame({
            'feature': X.columns,
            'mean_abs_shap': np.abs(shap_array).mean(axis=0)
        })
        shap_importance['field'] = msg_name
        # shap_importance = shap_importance.sort_values(by='mean_abs_shap', ascending=False)
        shap_importance['shap_ratio'] = shap_importance['mean_abs_shap'] / shap_importance['mean_abs_shap'].sum()
        shap_importance = shap_importance[['field', 'feature', 'shap_ratio']]

        # 중요도 기준 정렬
        shap_importance = shap_importance.sort_values(by='shap_ratio', ascending=False)

        # 누적 저장
        all_shap_results.append(shap_importance)

    except Exception as e:
        print(f"XXXXX {msg_name} 분석 실패: {e} XXXXX")
        continue

# 전체 통합 테이블 생성 및 저장
all_shap_df = pd.concat(all_shap_results, ignore_index=True)
os.makedirs("shap_results", exist_ok=True)
all_shap_df.to_csv("shap_results/all_fields_shap_importance_ratio.csv", index=False)
print("\n 모든 필드 SHAP 중요도 통합 저장 완료: shap_results/all_fields_shap_importance_ratio.csv")

all_model_df = pd.concat(all_model_results, ignore_index=True)
all_model_df.to_csv("shap_results/all_fields_model_comparison.csv", index=False)
print("\n 모든 필드 모델 성능 비교 결과 저장 완료: shap_results/all_fields_model_comparison.csv")




AHR2 Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.7880
et,Extra Trees Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3690
ada,Ada Boost Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,0.3560
gbc,Gradient Boosting Classifier,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,2.0460
dt,Decision Tree Classifier,0.9999,0.9999,0.9999,0.9999,0.9999,0.9998,0.9998,0.0430


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

ATT Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
et,Extra Trees Classifier,0.9993,1.0000,0.9989,0.9997,0.9993,0.9986,0.9986,0.2700
rf,Random Forest Classifier,0.9987,1.0000,0.9978,0.9996,0.9987,0.9973,0.9973,0.4990
dt,Decision Tree Classifier,0.9977,0.9977,0.9980,0.9974,0.9977,0.9954,0.9954,0.0380
gbc,Gradient Boosting Classifier,0.9931,0.9996,0.9921,0.9942,0.9932,0.9862,0.9862,0.7520
ada,Ada Boost Classifier,0.9830,0.9988,0.9856,0.9808,0.9832,0.9659,0.9660,0.1910


...

Processing:   0%|          | 0/25 [00:00<?, ?it/s]

PM Tree모델 학습 및 Best Model SHAP 분석


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9615,0.9875,0.9686,0.9567,0.9622,0.9230,0.9241,0.0140
gbc,Gradient Boosting Classifier,0.9547,0.9886,0.9685,0.9441,0.9557,0.9094,0.9107,0.0150
et,Extra Trees Classifier,0.9524,0.9895,0.9462,0.9601,0.9524,0.9049,0.9062,0.0270
rf,Random Forest Classifier,0.9513,0.9901,0.9552,0.9500,0.9518,0.9026,0.9042,0.0330
dt,Decision Tree Classifier,0.9400,0.9401,0.9305,0.9509,0.9396,0.8801,0.8819,0.0040


Processing:   0%|          | 0/25 [00:00<?, ?it/s]

XXXXX PM 분석 실패: Cannot take a larger sample than population when 'replace=False' XXXXX

 모든 필드 SHAP 중요도 통합 저장 완료: shap_results/all_fields_shap_importance_ratio.csv

 모든 필드 모델 성능 비교 결과 저장 완료: shap_results/all_fields_model_comparison.csv
